In [ ]:
import pandas as pd #pandas 불러오기
import numpy as np

# 벡터해준 데이터를 가져오는 과정
data = pd.read_json("./labeled_data.json")
data.columns = ["label", "trigram"]

In [1]:
# 삼 곱하기 오십 차원으로 바꿔주어 flastten 해준다는 과정이다
data['trigram'] = data['trigram'].apply(lambda x: (np.array(x).reshape(-1)))

1

In [2]:
# train데이터와 /테스트 data를 분리
from sklearn.model_selection import train_test_split #자모스플릿 가져오기
# data pop label 기준
y = data.pop('label')
X = data
# 엑스 트레인 데이터 와이 트레인 데이터 테스트 데이터 사이즈는 0.3 스테이트는 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
# keras 데이터 형식에 맞게 변환
X_train = np.array(X_train['trigram'].tolist())
X_test = np.array(X_test['trigram'].tolist())
y_train = y_train.values
y_test = y_test.values

In [4]:
X_train.shape # row 수를  x 임베딩 차원

(1283, 50)


In [5]:
y_train.shape # row개수 x 1

(112,3)


In [6]:
# reshape 과정
def reshape(df, dim): #케라스에 넣어주기 위함
    return df.reshape(df.shape[0], dim, 1)

In [7]:
X_train = reshape(X_train, 150) # x 트레인
X_test = reshape(X_test, 150) # x test
X_train.shape

In [8]:
# f1 score를 알아보고자 정의해주는 함수이다 
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
# 오픈 소스
def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
#precision_m
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.3)
from keras.models import Sequential
from keras.layers import Conv2D, Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import backend as K

K.clear_session() # session 초기화

model = Sequential()
model.add(Dense(150, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(BatchNormalization()) 
model.add(Conv1D(filters=100, kernel_size=5, activation='relu'))
model.add(Conv1D(filters=100, kernel_size=3, activation='sigmoid'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m])
model.summary()

In [10]:
import matplotlib.pyplot as plt

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.plot(epochs, history.history['val_f1_m'])
plt.title('model accuracy')
plt.ylabel('score')
plt.xlabel('epoch')
plt.legend(['train_acc', 'val_acc', 'f1score'], loc='upper left')
plt.savefig("./1DCNN reuslt.png")
plt.show()
# train에 대해서 overfitting. val은 조금 증가.
#epoch(iter)은 20정도가  충분하다.